In [ ]:
#Train the model(optional)
!pip install ultralytics
import os

from ultralytics import YOLO

model = YOLO("yolov8s.pt")
#replace with your yaml file
results = model.train(data=".yaml", epochs=62, hsv_h=0.02, hsv_s=0.2, hsv_v=0.4, flipud=0.5, fliplr=0.5, degrees=0.25, scale=0.3, translate=0.1, mosaic=0.5)

In [ ]:
#Train the Model option 2(optional)
!pip install ultralytics

import os
from ultralytics import YOLO

model = YOLO("yolov8m.pt")
#replace with your yaml file
results = model.train(data="/content/customdata1.yaml", epochs=80, imgsz=640, max_det=1000, batch=16, patience=20)

In [ ]:
#If you want to test the model on your own classified set
from ultralytics import YOLO
import torch
import pandas as pd
import os

# Load your trained model
model = YOLO('/content/last-15.pt')

# Function to detect estrous stage
def determine_estrus_stage(boxes, names):
    class_indices = boxes.data[:, -1].cpu().to(dtype=torch.int32).numpy()
    total_cells = len(class_indices)
    leukocytes_count = sum(1 for x in class_indices if names[x] == "Leukocyte")
    cornified_count = sum(1 for x in class_indices if names[x] == "Cornified")
    nucleated_count = sum(1 for x in class_indices if names[x] == "Nucleated")

    # Calculate exact percentages
    leukocytes_percentage = (leukocytes_count / total_cells) * 100 if total_cells else 0
    cornified_percentage = (cornified_count / total_cells) * 100 if total_cells else 0
    nucleated_percentage = (nucleated_count / total_cells) * 100 if total_cells else 0

    # Determine if high cell count
    is_high_cell_count = total_cells > 35

    # Initialize the predicted stage with a default value
    predicted_stage = "Metestrus"

    # Decision tree logic
    if is_high_cell_count:
        if leukocytes_percentage > 75:
            predicted_stage = "Diestrus"
        elif leukocytes_percentage >= 60 and cornified_percentage <= 10:
            predicted_stage = "Diestrus"
        elif cornified_percentage >= 70 and leukocytes_percentage <= 10:
            predicted_stage = "Estrus"
        elif nucleated_percentage > 40:
            predicted_stage = "Proestrus"
        else:
            predicted_stage = "Metestrus"
    else:  # Low cell count logic
        if leukocytes_count < 5:
            if cornified_percentage > nucleated_percentage:
                predicted_stage = "Estrus"
            elif nucleated_percentage > cornified_percentage:
                predicted_stage = "Proestrus"
            else:
                predicted_stage = "Metestrus"
        else:  # leukocytes_count >= 5
            if leukocytes_percentage > 70:
                predicted_stage = "Diestrus"
            elif nucleated_percentage >= cornified_percentage:
                predicted_stage = "Metestrus"
            else:
                predicted_stage = "Estrus"

    # Return the predicted stage along with the percentages
    return predicted_stage, leukocytes_percentage, cornified_percentage, nucleated_percentage

def process_image_and_get_stage(image_path):
    result = model(image_path, stream=True)
    for res in result:
        return determine_estrus_stage_and_display_percentages(res.boxes, res.names)

# Read the CSV file with correct labels
df_correct = pd.read_csv("")# Replace with your CSV file name

# Directory containing images
image_directory = ''

# Track accuracy and mismatches
accurate_predictions = 0
mismatches = []

# Compare model output with correct labels
for _, row in df_correct.iterrows():
    image_number = row['ImageNumber']
    true_stage = row['Estrous Stage']
    image_path = os.path.join(image_directory, f"{image_number}.tif")
    
    # Check if the image file exists
    if os.path.exists(image_path):
        predicted_stage = process_image_and_get_stage(image_path)

        if predicted_stage == true_stage:
            accurate_predictions += 1
        else:
            mismatches.append({'ImageNumber': image_number,
                            'Predicted_Stage': predicted_stage,
                            'Correct_Stage': true_stage})

# Calculate accuracy
accuracy = (accurate_predictions / len(df_correct)) * 100
print(f"Model accuracy: {accuracy}%")

# Display and save incorrect images information
if mismatches:
    df_mismatches = pd.DataFrame(mismatches)
    print("Incorrectly predicted images:")
    print(df_mismatches)
    df_mismatches.to_csv('mismatches.csv', index=False)

In [ ]:
# Classification
!pip install ultralytics

# Cell 2
from ultralytics import YOLO
import torch
import pandas as pd
import os

# Load your trained model
model = YOLO('/content/last-15.pt')

# Function to detect estrous stage
def determine_estrus_stage(boxes, names):
    class_indices = boxes.data[:, -1].cpu().to(dtype=torch.int32).numpy()
    total_cells = len(class_indices)
    leukocytes_count = sum(1 for x in class_indices if names[x] == "Leukocyte")
    cornified_count = sum(1 for x in class_indices if names[x] == "Cornified")
    nucleated_count = sum(1 for x in class_indices if names[x] == "Nucleated")

    # Calculate exact percentages
    leukocytes_percentage = (leukocytes_count / total_cells) * 100 if total_cells else 0
    cornified_percentage = (cornified_count / total_cells) * 100 if total_cells else 0
    nucleated_percentage = (nucleated_count / total_cells) * 100 if total_cells else 0

    # Determine if high cell count
    is_high_cell_count = total_cells > 35

    # Initialize the predicted stage with a default value
    predicted_stage = "Metestrus"

    # Decision tree logic
    if is_high_cell_count:
        if leukocytes_percentage > 75:
            predicted_stage = "Diestrus"
        elif leukocytes_percentage >= 60 and cornified_percentage <= 10:
            predicted_stage = "Diestrus"
        elif cornified_percentage >= 70 and leukocytes_percentage <= 10:
            predicted_stage = "Estrus"
        elif nucleated_percentage > 40:
            predicted_stage = "Proestrus"
        else:
            predicted_stage = "Metestrus"
    else:  # Low cell count logic
        if leukocytes_count < 5:
            if cornified_percentage > nucleated_percentage:
                predicted_stage = "Estrus"
            elif nucleated_percentage > cornified_percentage:
                predicted_stage = "Proestrus"
            else:
                predicted_stage = "Metestrus"
        else:  # leukocytes_count >= 5
            if leukocytes_percentage > 70:
                predicted_stage = "Diestrus"
            elif nucleated_percentage >= cornified_percentage:
                predicted_stage = "Metestrus"
            else:
                predicted_stage = "Estrus"

    # Return the predicted stage along with the percentages
    return predicted_stage, leukocytes_percentage, cornified_percentage, nucleated_percentage

# Cell 3
def process_image_and_get_stage(image_path):
    result = model.predict(image_path, conf=0.3, max_det=1000, stream=True)
    for res in result:
        return determine_estrus_stage(res.boxes, res.names)

# Cell 4
# Directory containing the images
images_directory = '/content/drive/MyDrive/Object Detection/TEST'

# List to store results
results = []

# Loop through each image in the directory and process
for image_name in os.listdir(images_directory):
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(images_directory, image_name)
        predicted_stage, leukocytes_percentage, cornified_percentage, nucleated_percentage, low_cell_flag = process_image_and_get_stage(image_path)

        # Append result for current image
        result = {
            'ImageName': image_name,
            'Leukocytes %': leukocytes_percentage,
            'Cornified %': cornified_percentage,
            'Nucleated %': nucleated_percentage,
            'Predicted_Stage': predicted_stage,
            'Low_Cell_Count': low_cell_flag
        }
        results.append(result)

        # Print result for current image
        print(f"Processed: {image_name}")

# Cell 5
# Save the results after processing all images
column_order = ['ImageName', 'Leukocytes %', 'Cornified %', 'Nucleated %', 'Predicted_Stage', 'Low_Cell_Count']
df_results = pd.DataFrame(results, columns=column_order)
df_results.to_csv('/content/results.csv', index=False)

# Optionally, print the entire DataFrame after all images are processed
print("\nAll processed images:")
print(df_results)

In [ ]:
#SINGLE IMAGE DISPLAY Of PREDICTIONS

# Import necessary libraries
from ultralytics import YOLO
import numpy as np
from PIL import Image
import cv2
import torch

# Check if the script is running in Google Colab
try:
    from google.colab.patches import cv2_imshow
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Load a pretrained YOLOv8 model
model = YOLO('/content/last-15.pt')

# Open the local image and convert it to a numpy array
image_path = ''
image = Image.open(image_path).convert("RGB")
image_np = np.array(image)

# Use the numpy array for prediction
results = model.predict(image)

def plot_bboxes(image, boxes, labels=[], colors=[], score=True, conf=None):
    # Check if image is a PIL Image and convert it to numpy array
    if isinstance(image, Image.Image):
        image = np.array(image)

    # Define custom labels and colors if not provided
    if not labels:
        labels = {0: 'Leukocyte', 1: 'Cornified', 2: 'Nucleated'}
    if not colors:
        colors = [(89, 161, 197), (67, 161, 255), (19, 222, 24)]

    # Plot each box
    for box in boxes:
        # Add score in label if score is True
        label = labels[int(box[-1])]
        if score:
            label += f" {round(100 * float(box[-2]), 1)}%"

        # Filter every box under conf threshold if conf threshold is set
        if conf and box[-2] <= conf:
            continue

        color = colors[int(box[-1])]
        box_label(image, box, label, color)

    # Convert image to RGB and display
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if IN_COLAB:
        cv2_imshow(image)
    else:
        cv2.imshow("Image", image)

# Function to draw boxes and labels on the image
def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
    lw = max(round(sum(image.shape) / 2 * 0.0005), 2)  # Line width
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))  # Box corners
    cv2.rectangle(image, p1, p2, color, thickness=lw, lineType=cv2.LINE_AA)  # Draw rectangle

    if label:
        tf = max(lw - 1, 1)  # Font thickness
        w, h = cv2.getTextSize(label, 0, fontScale=lw / 3, thickness=tf)[0]  # Text width and height
        outside = p1[1] - h >= 3
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)  # Draw filled rectangle
        cv2.putText(image, label, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                    0, lw / 3, txt_color, thickness=tf, lineType=cv2.LINE_AA)  # Draw label text
def determine_estrus_stage(boxes, names):
    class_indices = boxes.data[:, -1].cpu().to(dtype=torch.int32).numpy()
    total_cells = len(class_indices)
    leukocytes_count = sum(1 for x in class_indices if names[x] == "Leukocyte")
    cornified_count = sum(1 for x in class_indices if names[x] == "Cornified")
    nucleated_count = sum(1 for x in class_indices if names[x] == "Nucleated")

    # Calculate exact percentages
    leukocytes_percentage = (leukocytes_count / total_cells) * 100 if total_cells else 0
    cornified_percentage = (cornified_count / total_cells) * 100 if total_cells else 0
    nucleated_percentage = (nucleated_count / total_cells) * 100 if total_cells else 0

    # Determine if high cell count
    is_high_cell_count = total_cells > 35

    # Initialize the predicted stage with a default value
    predicted_stage = "Metestrus"

    # Decision tree logic
    if is_high_cell_count:
        if leukocytes_percentage > 75:
            predicted_stage = "Diestrus"
        elif leukocytes_percentage >= 60 and cornified_percentage <= 10:
            predicted_stage = "Diestrus"
        elif cornified_percentage >= 70 and leukocytes_percentage <= 10:
            predicted_stage = "Estrus"
        elif nucleated_percentage > 40:
            predicted_stage = "Proestrus"
        else:
            predicted_stage = "Metestrus"
    else:  # Low cell count logic
        if leukocytes_count < 5:
            if cornified_percentage > nucleated_percentage:
                predicted_stage = "Estrus"
            elif nucleated_percentage > cornified_percentage:
                predicted_stage = "Proestrus"
            else:
                predicted_stage = "Metestrus"
        else:  # leukocytes_count >= 5
            if leukocytes_percentage > 70:
                predicted_stage = "Diestrus"
            elif nucleated_percentage >= cornified_percentage:
                predicted_stage = "Metestrus"
            else:
                predicted_stage = "Estrus"

    # Return the predicted stage along with the percentages
    return predicted_stage, leukocytes_percentage, cornified_percentage, nucleated_percentage

stage = detect_estrus_stage(results)
print(f"\nThe stage of the estrous cycle is {stage}")

# Display predictions with bounding boxes
plot_bboxes(image, results[0].boxes.data, conf=0.1)